<a href="https://colab.research.google.com/github/dgromann/cl_intro_ws2024/blob/main/exercises/HomeExercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home Exericse 3: Hyperparameters and Evaluation
In this third home exercise, you will use the knowledge from Tutorial 4 to experiment with hyperparameters, create a test set, and evaluate your final model on the created test set.

In this notebook, please complete all instructions starting with 👋 ⚒ in the code cell after the sign or provide your analysis in the text cell after the sign.

## **Distilbert: Hyperparameters and Evaluation**

Use the code of Tutorial 4 to load and fine-tune the `distilbert-base-cased`model on the small subset of the `imdb`Movie Review Dataset. For convenience, the code of Tutorial 4 required for this exercise is already provided in the code cells below.

👋 ⚒ When creating the dataset splits in the code cell below, additionally create a test set to be used after thet training. Make sure that your test set does not contain any of the sentences contained in the training or validation set and is approximately of the same size as the validation set.

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate --upgrade

  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
Using cached evaluate-0.4.3-py3-none-any.whl (84 kB)


In [4]:
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorWithPadding, AutoTokenizer

imdb_dataset = load_dataset("imdb")
# we had loaded the imdb dataset already above - if not, outcomment this line
# Make sure you have the right tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")

# Just take the first 50 tokens for speed on CPU
def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:512]),
        'label': example['label']
    }

train_indices = range(0, 10000)
val_indices = range(10000, 11000)
test_indices = range(11000, 12000)

# Take 128 random examples for train and 32 validation
small_imdb_dataset = DatasetDict(
    train=imdb_dataset['train'].shuffle(seed=24).select(train_indices).map(truncate),
    val=imdb_dataset['train'].shuffle(seed=24).select(val_indices).map(truncate),
    test=imdb_dataset['train'].shuffle(seed=24).select(test_indices).map(truncate)
)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

small_tokenized_dataset = small_imdb_dataset.map(tokenize_function, batched=True, batch_size=16)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

👋 ⚒ For this exercise, we will use the Hugging Face Trainer class to play with hyperparamters. Try to find a set of hyperparameter settings that achieves the highest possilbe accuracy on the **validation set** with the small dataset and model in this setup.

**Optional:** If you want to follow a more systematic route, feel free to use available frameworks for hyperparameter optimization, such as [Optuna](https://optuna.org/).

In [11]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
import optuna

model = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=2)
accuracy = evaluate.load("accuracy")

# Define the Optuna objective
def objective(trial):
    learning_rate = trial.suggest_float('learning_rate', 2e-5, 5e-5, log=True)  
    weight_decay = trial.suggest_float('weight_decay', 0.01, 0.1, log=True) 
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32]) 
    num_epochs = trial.suggest_int('num_epochs', 2, 10)

    training_args = TrainingArguments(
        output_dir=f"optuna_distilbert_{trial.number}",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        logging_steps=8,
        num_train_epochs=num_epochs,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        load_best_model_at_end=True,
        save_total_limit=1,
        report_to= 'none',
        seed=224
    )

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return accuracy.compute(predictions=predictions, references=labels)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=small_tokenized_dataset['train'],
        eval_dataset=small_tokenized_dataset['val'],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    trainer.train()

    eval_metrics = trainer.evaluate()
    return eval_metrics['eval_accuracy']

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

best_trial = study.best_trial
print(f"Best trial hyperparameters: {best_trial.params}")

best_model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-cased', num_labels=2
)
best_training_args = TrainingArguments(
    output_dir="best_model",
    per_device_train_batch_size=best_trial.params['batch_size'],
    per_device_eval_batch_size=best_trial.params['batch_size'],
    logging_steps=8,
    num_train_epochs=best_trial.params['num_epochs'],
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_trial.params['learning_rate'],
    weight_decay=best_trial.params['weight_decay'],
    load_best_model_at_end=True,
    save_total_limit=1,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return accuracy.compute(predictions=predictions, references=labels)

best_trainer = Trainer(
    model=best_model,
    args=best_training_args,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

best_trainer.train()

best_trainer.save_model("best_model")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2024-11-29 13:01:03,460] A new study created in memory with name: no-name-6492eb38-1539-4596-9065-b28989f10b62


Epoch,Training Loss,Validation Loss,Accuracy
1,0.302200,0.302230,0.880000
2,0.379600,0.445726,0.874000
3,0.059900,0.575594,0.880000
4,0.031900,0.724912,0.893000
5,0.000700,0.694777,0.885000
6,0.000200,0.757625,0.892000
7,0.000100,0.847184,0.892000


[I 2024-11-29 13:33:44,973] Trial 0 finished with value: 0.88 and parameters: {'learning_rate': 4.910245840098907e-05, 'weight_decay': 0.02332477955536842, 'batch_size': 8, 'num_epochs': 7}. Best is trial 0 with value: 0.88.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.183700,0.407537,0.883000
2,0.090500,0.446379,0.886000
3,0.004700,0.566851,0.878000
4,0.016800,0.644274,0.883000
5,0.018800,0.650853,0.890000
6,0.001200,0.693659,0.888000
7,0.017900,0.784914,0.891000
8,0.000300,0.821126,0.887000
9,0.000400,0.816140,0.887000
10,0.000400,0.810726,0.890000


[I 2024-11-29 14:15:37,346] Trial 1 finished with value: 0.883 and parameters: {'learning_rate': 3.505391372702253e-05, 'weight_decay': 0.049926089086037524, 'batch_size': 32, 'num_epochs': 10}. Best is trial 1 with value: 0.883.


...

[I 2024-11-30 00:44:39,060] Trial 28 finished with value: 0.837 and parameters: {'learning_rate': 3.7767426366821304e-05, 'weight_decay': 0.030118566625400255, 'batch_size': 16, 'num_epochs': 6}. Best is trial 9 with value: 0.888.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.000000,2.128082,0.845000
2,0.000000,2.372188,0.840000
3,0.000000,2.426211,0.845000
4,0.000000,2.458859,0.840000
5,0.000000,2.866641,0.838000
6,0.000000,2.481660,0.844000
7,0.000000,2.590931,0.838000
8,0.000000,2.579707,0.840000


[I 2024-11-30 01:18:28,570] Trial 29 finished with value: 0.845 and parameters: {'learning_rate': 3.427170036598393e-05, 'weight_decay': 0.02577444189414341, 'batch_size': 32, 'num_epochs': 8}. Best is trial 9 with value: 0.888.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Best trial hyperparameters: {'learning_rate': 3.277703685345742e-05, 'weight_decay': 0.012488808024735532, 'batch_size': 32, 'num_epochs': 10}


Epoch,Training Loss,Validation Loss,Accuracy
1,0.359200,0.303364,0.872000
2,0.189000,0.331404,0.886000
3,0.102500,0.404461,0.895000
4,0.055500,0.529380,0.887000
5,0.011400,0.600677,0.882000
6,0.007100,0.730457,0.883000
7,0.002400,0.794763,0.888000
8,0.029500,0.821378,0.884000
9,0.000200,0.803798,0.887000
10,0.000200,0.798478,0.888000


👋 ⚒ Change the following code cell in a way that not only a single sentence is evaluated on your trained model (!make sure to use the correct checkpoint!) but the evaluation is performaned on the entire newly created test set.

This might also be a good occassion to get familiar with the [Hugging Face documentation and tutorials](https://huggingface.co/docs/transformers/index).

In [24]:
import torch
import numpy as np
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer
from sklearn.metrics import matthews_corrcoef, accuracy_score, precision_recall_fscore_support

imdb_dataset = load_dataset("imdb")

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:512]),
        'label': example['label']
    }

train_indices = range(0, 10000)
val_indices = range(10000, 11000)
test_indices = range(11000, 12000)

small_imdb_dataset = DatasetDict({
    "train": imdb_dataset['train'].shuffle(seed=24).select(train_indices).map(truncate),
    "val": imdb_dataset['train'].shuffle(seed=24).select(val_indices).map(truncate),
    "test": imdb_dataset['train'].shuffle(seed=24).select(test_indices).map(truncate)
})

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

small_tokenized_dataset = small_imdb_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(
    "/Users/bohdan/Documents/cl_intro_ws2024_zhvalevskyi/exercises/best_model"
)

trainer = Trainer(
    model=fine_tuned_model,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    mcc = matthews_corrcoef(labels, preds)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")

    return {
        "mcc": mcc,
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

test_results = trainer.evaluate(small_tokenized_dataset["test"], metric_key_prefix="test")
predictions = trainer.predict(small_tokenized_dataset["test"])

metrics = compute_metrics(predictions)

print("Evaluation Results:")
print(f"Test Loss: {test_results['test_loss']:.4f}")
print(f"Matthews Correlation Coefficient (MCC): {metrics['mcc']:.4f}")
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"F1 Score: {metrics['f1']:.4f}")


Evaluation Results:
Test Loss: 0.2870
Matthews Correlation Coefficient (MCC): 0.7764
Accuracy: 0.8830
Precision: 0.9560
Recall: 0.8082
F1 Score: 0.8759
